In [ ]:
# this code calculate all the positions that can have 
# synonymous mutations/non-synonymous mutations in known protein coding genes(same as in coverage)
# Author: Yiyun


# import sys  # In case not appended
# sys.path
# sys.path.append('/storage/home/yur97/work/anaconda3/lib/python3.8/site-packages')

In [2]:
import pandas as pd
import os
import pickle
from tqdm import tqdm
from Bio import SeqIO
from Bio.Seq import Seq
from utils import *
from multiprocessing import Pool

In [3]:
### Load annotation data
dict_name = pickle.load(open('../proc_09152020/dict_name.pkl','rb'))
dict_transcript_info = pickle.load(open('../proc_09152020/dict_transcript_info.pkl','rb'))
dict_record = pickle.load(open('../proc_09152020/dict_record_new.pkl','rb'))
# dict_record = SeqIO.to_dict(SeqIO.parse('../anno_ref/gencode_v19/GRCh37.p13.genome.fa', 'fasta'))
# pickle.dump(dict_record, open('../proc_09152020/dict_record_new.pkl','wb')) # new dict record that compatible with python3

In [4]:
# Get number of positions that can have silent mutations in a known protein coding gene
def get_n(cds_seq):
    base_context = None   #codon
    context_position = None  #position of base in codon
    lp_flag = 0  #low quality flag
    n_context = 0  #cds position
    silent_bases = 0 # Total silent positions in the gene
    nonsilent_bases = 0

    for bases in cds_seq:
        silent_n = 0 # if a base can cause silent mutation or not

        if bases == "N":
            lp_flag = 1
            continue

        #Determine codon context
        n_context += 1
        if n_context % 3 != 0:
            context_position = n_context % 3 - 1
            codon_n = (n_context // 3) * 3
            base_context = cds_seq[codon_n:codon_n + 3]

        else:
            context_position = 2
            codon_n = n_context
            base_context = cds_seq[codon_n - 3:codon_n]
        #If any mutation can cause silent mutation, if can, change flag
        other_nuc = filter(lambda i: i != bases, nuc)
        for mut_bases in other_nuc:
            context_before = base_context  # This is the original tri-nucleotide context
            context_after = base_context[:context_position] + mut_bases + base_context[context_position + 1:]
            if is_silent(context_before, context_after):
                silent_n += float(1/3)

        # From silent_n for this base get nonsilent_n for this base
        nonsilent_n = 1-silent_n
        
        # get silent bases and nonsilent bases
        silent_bases+= silent_n
        nonsilent_bases+=nonsilent_n
    
    return [silent_bases, nonsilent_bases]

In [5]:
# Define function for multiprocessing
def cal_n_silent(transcript):
#     name = dict_name[transcript]
    strand = dict_transcript_info[transcript]['strand']
    chromosome = dict_transcript_info[transcript]['chr'].strip('chr')
    list_cds = get_mrna_position(transcript, dict_transcript_info, strand)
    seq_transcript = get_transcript_sequence(transcript, dict_transcript_info, dict_record, strand)
    seq_cds = get_cdna_sequence(list_cds, strand, seq_transcript)
    try:
        res = get_n(seq_cds)
    except KeyError:
        # If the cds is not complete, we directly use 1/3 of the sequence length
        res = [len(seq_cds)/3, (len(seq_cds)/3)*2]
        
    return res

In [6]:
# Multiprocessing calculate all synonymous positions in genes
p = Pool(4)
res = p.map(cal_n_silent, dict_transcript_info.keys())
p.close()
p.join()

In [7]:
syn = [];nsyn = []
for transcripts in res:
    syn.append(transcripts[0])
    nsyn.append(transcripts[1])

In [9]:
round(sum(syn))

7564562

In [8]:
round(sum(nsyn))

25388970